# Catboost

1. Построить ансамбль решающих деревьев используя catboost
2. Используя перекрестную проверку найти наилучшие параметры
3. Сделать предсказание и проверить качество через каппа-метрику

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn import preprocessing

from core.reduce_mem_usage import reduce_mem_usage

## Загрузка данных

In [2]:
data = pd.read_csv("../data/prudential/train.csv.gz")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


## Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)

In [4]:
list_df = [data]
for un in data["Product_Info_2_1"].unique():
    s = pd.DataFrame()
    s["Product_Info_2_1_" + un] = data["Product_Info_2_1"].isin([un]).astype("int8")
    list_df.append(s)
data = pd.concat(list_df, axis=1)
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)

In [5]:
data.fillna(value=-1, inplace=True)
data["Response"] = data["Response"] - 1
print(data.head())

   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   2               1              10        0.076923               2   
1   5               1              26        0.076923               2   
2   6               1              26        0.076923               2   
3   7               1              10        0.487179               2   
4   8               1              26        0.230769               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               1               1  0.641791  0.581818  0.148536  ...   
1               3               1  0.059701  0.600000  0.131799  ...   
2               3               1  0.029851  0.745455  0.288703  ...   
3               3               1  0.164179  0.672727  0.205021  ...   
4               3               1  0.417910  0.654545  0.234310  ...   

   Medical_Keyword_46  Medical_Keyword_47  Medical_Keyword_48  Response  \
0                   0                   0            

## Набор столбцов для расчета

In [6]:
columns_groups = [
    "Insurance_History",
    "InsuredInfo",
    "Medical_Keyword",
    "Family_Hist",
    "Medical_History",
    "Product_Info",
]
columns = [
    "Wt",
    "Ht",
    "Ins_Age",
    "BMI",
]

for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print(columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keywor

## Нормализация данных

In [7]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data, columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

In [8]:
data_transformed = reduce_mem_usage(data_transformed)
print(data_transformed.info())

Потребление памяти меньше на - 42.87 Мб (минус 75.1%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 126 entries, 0 to Response
dtypes: float16(125), int8(1)
memory usage: 14.2 MB
None


## Разделить данные

In [9]:
data_train, data_test = train_test_split(data_transformed, test_size=0.2)
print(data_train.head())

              0         1         2         3         4         5         6  \
46300  0.825684 -1.200195 -0.467285  2.105469  0.611816 -0.169434 -1.159180   
37302  4.257812  1.249023 -0.467285  3.933594  0.611816 -0.169434 -1.159180   
230   -1.571289 -0.220581 -0.391602 -1.843750  0.611816 -0.169434 -1.159180   
18358  0.426270  1.003906 -0.845703 -0.081970  0.611816 -0.169434  0.862305   
21506 -0.983398 -0.220581  0.062622 -1.086914  0.611816 -0.169434  0.862305   

              7         8         9  ...       116       117       118  \
46300  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
37302  1.100586 -1.156250  1.130859  ... -0.083679 -2.263672 -0.149292   
230    1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
18358 -1.013672  0.880859 -0.928711  ... -0.083679  0.441650 -0.149292   
21506  0.043671  0.863770 -0.928711  ... -0.083679  0.441650 -0.149292   

            119       120       121       122       123      124  Response  
463

## CatBoost

In [10]:
x = pd.DataFrame(data_train, columns=columns_transformed)
train_dataset = Pool(data=x, label=data_train["Response"])
model = CatBoostClassifier(
    iterations=10,
    learning_rate=0.57,
    random_seed=17,
    depth=6,
    loss_function="MultiClass",
    bootstrap_type="MVS",
    custom_metric="WKappa",
)

In [11]:
cb_params = {
    "depth": range(5, 8),
    "learning_rate": np.arange(0.56, 0.59, 0.01),
    "l2_leaf_reg": range(1, 5)
}

cb_grid = model.grid_search(cb_params, cv=5, X=x, y=data_train["Response"], verbose=True)


0:	learn: 1.5690290	test: 1.5694284	best: 1.5694284 (0)	total: 332ms	remaining: 2.99s
1:	learn: 1.4701038	test: 1.4685465	best: 1.4685465 (1)	total: 430ms	remaining: 1.72s
2:	learn: 1.4268608	test: 1.4286601	best: 1.4286601 (2)	total: 517ms	remaining: 1.21s
3:	learn: 1.3430713	test: 1.3445567	best: 1.3445567 (3)	total: 635ms	remaining: 952ms
4:	learn: 1.3188124	test: 1.3222678	best: 1.3222678 (4)	total: 808ms	remaining: 808ms
5:	learn: 1.3013819	test: 1.3081467	best: 1.3081467 (5)	total: 917ms	remaining: 611ms
6:	learn: 1.2884617	test: 1.2946376	best: 1.2946376 (6)	total: 1.07s	remaining: 457ms
7:	learn: 1.2799046	test: 1.2893624	best: 1.2893624 (7)	total: 1.23s	remaining: 307ms
8:	learn: 1.2730142	test: 1.2830789	best: 1.2830789 (8)	total: 1.35s	remaining: 150ms
9:	learn: 1.2647180	test: 1.2758564	best: 1.2758564 (9)	total: 1.45s	remaining: 0us

bestTest = 1.275856408
bestIteration = 9

0:	loss: 1.2758564	best: 1.2758564 (0)	total: 2.13s	remaining: 1m 14s
0:	learn: 1.5679650	test: 1.5

## Самые оптимальные параметры

In [12]:
print(cb_grid["params"])

{'depth': 7, 'l2_leaf_reg': 4, 'learning_rate': 0.5800000000000001}


## Итоговая модель

In [13]:
model = CatBoostClassifier(
    iterations=100,
    learning_rate=cb_grid["params"]["learning_rate"],
    random_seed=17,
    depth=cb_grid["params"]["depth"],
    l2_leaf_reg=cb_grid["params"]["l2_leaf_reg"],
    loss_function="MultiClass",
    bootstrap_type="MVS",
    custom_metric="WKappa",
)

In [14]:
model.fit(train_dataset)

0:	learn: 1.5418458	total: 557ms	remaining: 55.2s
1:	learn: 1.4048778	total: 1.17s	remaining: 57.5s
2:	learn: 1.3547880	total: 1.7s	remaining: 55s
3:	learn: 1.3213897	total: 2.18s	remaining: 52.2s
4:	learn: 1.3034784	total: 2.86s	remaining: 54.3s
5:	learn: 1.2735610	total: 3.37s	remaining: 52.9s
6:	learn: 1.2581857	total: 3.96s	remaining: 52.7s
7:	learn: 1.2431048	total: 4.9s	remaining: 56.4s
8:	learn: 1.2334102	total: 5.47s	remaining: 55.4s
9:	learn: 1.2259894	total: 5.87s	remaining: 52.8s
10:	learn: 1.2170435	total: 6.25s	remaining: 50.6s
11:	learn: 1.2092700	total: 6.63s	remaining: 48.6s
12:	learn: 1.2035381	total: 7s	remaining: 46.9s
13:	learn: 1.1997660	total: 7.38s	remaining: 45.4s
14:	learn: 1.1943182	total: 7.76s	remaining: 44s
15:	learn: 1.1887979	total: 8.14s	remaining: 42.8s
16:	learn: 1.1802083	total: 8.54s	remaining: 41.7s
17:	learn: 1.1762628	total: 8.91s	remaining: 40.6s
18:	learn: 1.1735314	total: 9.29s	remaining: 39.6s
19:	learn: 1.1690832	total: 9.66s	remaining: 38.7s

## Предсказание данных

In [15]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
data_test["target"] = model.predict(Pool(data=x_test))

## Оценка модели

In [16]:
print(f'CatBoost: {round(cohen_kappa_score(data_test["target"], data_test["Response"], weights="quadratic"), 3)}')

CatBoost: 0.553


## Матрица неточностей

In [17]:
print(f'CatBoost:\n\n {confusion_matrix(data_test["target"], data_test["Response"])}')

CatBoost:

 [[ 301  179   27   18   54  125   50   30]
 [ 205  344   11    3  113  101   43   25]
 [  18   17   74   13    0    1    0    0]
 [  31   20   61  176    0    3    0    3]
 [  97  127    7    0  611  125   13    9]
 [ 197  254   12   17  209 1181  242  133]
 [ 126  110    5   11   43  310  699  185]
 [ 268  217    6   40  103  409  586 3479]]
